### Install transformers library

In [ ]:
!pip install transformers

     |████████████████████████████████| 1.8MB 5.0MB/s 
     |████████████████████████████████| 890kB 30.4MB/s 
     |████████████████████████████████| 2.9MB 39.2MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893261 sha256=2ab427167bebb5545c6a1917110b0999dea116ed90f503c254bccdb7f44eb3b7
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


### Essential libraries imported

In [ ]:
import torch
from transformers import AutoTokenizer,BertTokenizerFast,BertForQuestionAnswering
from google.colab import drive
drive.mount("/content/drive") 

Mounted at /content/drive


### Prepare GPU Cuda.

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Device available for running: ")
print(device)

Device available for running: 
cpu


### Define tokenizer and load fine tuned model 

In [ ]:
tokenizer = BertTokenizerFast.from_pretrained('bert-base-uncased')
# tokenizer = AutoTokenizer.from_pretrained('bert-base-uncased')

model = BertForQuestionAnswering.from_pretrained('bert-large-uncased-whole-word-masking-finetuned-squad').to(device)
model.eval()

BertForQuestionAnswering(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 1024, padding_idx=0)
      (position_embeddings): Embedding(512, 1024)
      (token_type_embeddings): Embedding(2, 1024)
      (LayerNorm): LayerNorm((1024,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=1024, out_features=1024, bias=True)
              (key): Linear(in_features=1024, out_features=1024, bias=True)
              (value): Linear(in_features=1024, out_features=1024, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=1024, out_features=1024, bias=True)
              (LayerNorm): LayerNorm((1024,), eps=1e-12,

### Declare some functions from official evaluation script of SQuAD 2.0 Dataset so as to evaluate model's answers

In [ ]:
def get_prediction(context,question):

  inputs = tokenizer.encode_plus(question, context, return_tensors='pt').to(device)

  outputs = model(**inputs)
  answer_start = torch.argmax(outputs[0])  
  answer_end = torch.argmax(outputs[1]) + 1 

  answer = tokenizer.convert_tokens_to_string(tokenizer.convert_ids_to_tokens(inputs['input_ids'][0][answer_start:answer_end]))

  return answer

def normalize_text(s):
  """Removing articles and punctuation, and standardizing whitespace are all typical text processing steps."""
  import string, re

  def remove_articles(text):
    regex = re.compile(r"\b(a|an|the)\b", re.UNICODE)
    return re.sub(regex, " ", text)

  def white_space_fix(text):
    return " ".join(text.split())

  def remove_punc(text):
    exclude = set(string.punctuation)
    return "".join(ch for ch in text if ch not in exclude)

  def lower(text):
    return text.lower()

  return white_space_fix(remove_articles(remove_punc(lower(s))))

def compute_exact_match(prediction, truth):
    return int(normalize_text(prediction) == normalize_text(truth))

def compute_f1(prediction, truth):
  pred_tokens = normalize_text(prediction).split()
  truth_tokens = normalize_text(truth).split()
  
  # if either the prediction or the truth is no-answer then f1 = 1 if they agree, 0 otherwise
  if len(pred_tokens) == 0 or len(truth_tokens) == 0:
    return int(pred_tokens == truth_tokens)
  
  common_tokens = set(pred_tokens) & set(truth_tokens)
  
  # if there are no common tokens then f1 = 0
  if len(common_tokens) == 0:
    return 0
  
  prec = len(common_tokens) / len(pred_tokens)
  rec = len(common_tokens) / len(truth_tokens)
  
  return 2 * (prec * rec) / (prec + rec)
  
def query_answer(context,question,answer):

  prediction = get_prediction(context,question)
  em_score = compute_exact_match(prediction, answer)
  f1_score = compute_f1(prediction, answer)

  print(f"Question: {question}")
  print(f"Prediction: {prediction}")
  print(f"True Answer: {answer}")
  print(f"EM: {em_score} \t F1: {f1_score}\n")

```
> In this cell, we built some functions (with help of official evaluation script of SQuAD v2, in that way to give each 
time a paragraph (context in which at most times answer is included), a specific question and it's real answer!
```

From [Question Answering on SQuAD 2.0 Dataset](https://web.stanford.edu/class/archive/cs/cs224n/cs224n.1194/reports/default/15848195.pdf) paper we retrieve more information about evaluation metrics:
```
Two evaluation metrics are employed: Exact Match (EM) score and F1 score. EM is a binary measurement of whether the 
percentage of output from a system exactly matches the ground truth answer(the proportion of questions that are 
answered in exact same words as the ground truth). F1 score is a harmonic mean of precision an recall. For each 
question, precision is calculated as the number of correctly predicted words divided by the total words in the predicted 
answer. Recall is the number of correctly predicted words divided by the number of words in the ground truth answer.
The F1 score is averaged among questions
```

### First example with Carles Puyol

In [ ]:
context = """Carles Puyol Saforcada, born 13 April 1978, is a Spanish retired professional footballer
             who played his entire career for Barcelona. Mainly a central defender, he could also play in either full-back position, mostly as a right-back, and 
             is regarded as one of the best defenders of his generation and all time. He was Barcelona's captain from August 2004 until his retirement in 2014, 
             and appeared in 593 competitive matches for the club. He won 18 major club titles, including six La Liga trophies and three Champions Leagues.
             Puyol won 100 caps for Spain, and was part of the squads that won Euro 2008 and the 2010 World Cup. In the 2010 World Cup semi-final, he scored 
             the only goal of the game against Germany.
          """

questions = [
             "How many caps did Puyol win for Spain?",
             "How many matches Puyol appeared?",
             "How many matches Puyol played?",
             "When did Puyol retire?"]
answers = ["100","593","593","2014"]

for q,a in zip(questions,answers):
  query_answer(context,q,a)

Question: How many caps did Puyol win for Spain?
Prediction: 100
True Answer: 100
EM: 1 	 F1: 1.0

Question: How many matches Puyol appeared?
Prediction: 593
True Answer: 593
EM: 1 	 F1: 1.0

Question: How many matches Puyol played?
Prediction: 593
True Answer: 593
EM: 1 	 F1: 1.0

Question: When did Puyol retire?
Prediction: 2014
True Answer: 2014
EM: 1 	 F1: 1.0



> Ηere we observe that pretrained fine tuned model did amazing work, as our model that we fine tuned did! So until now no differences exist!

### Second example with Ioannina city

In [ ]:
context = """Greece, officially the Hellenic Republic and also known as Hellas, is a country located in Southeast Europe.
             Its population is approximately 10.7 million as of 2018; Athens is its largest and capital city, followed by Thessaloniki.
             Situated on the southern tip of the Balkans, Greece is located at the crossroads of Europe, Asia, and Africa. 
             Another beautiful city which is located in the northwest part of Greece is Ioannina with population 100 thousand people. 
          """

questions = ["What's population of Ioannina?","Where is Ioannina located?","What is Ioannina?"]
answers = ["100 thousand","in the northwest part of Greece","city"]

for q,a in zip(questions,answers):
  query_answer(context,q,a)

Question: What's population of Ioannina?
Prediction: 100 thousand
True Answer: 100 thousand
EM: 1 	 F1: 1.0

Question: Where is Ioannina located?
Prediction: northwest part of greece
True Answer: in the northwest part of Greece
EM: 0 	 F1: 0.888888888888889

Question: What is Ioannina?
Prediction: another beautiful city which is located in the northwest part of greece
True Answer: city
EM: 0 	 F1: 0.16666666666666669



Here we have 2 main differences in answers of 2 last questions! 

1. First answer was correct by both models-. 
 
2. In second question fine-tuned-large-bert model as we already referred performed better, cause it returned more accurate answer, 'northwest part of Greece' instead of 'northwest part', which is a huge improvement. 

3. Also, in third question suprisingly our model, that we fine tuned was concise, with answer 'beautiful city' instead of 'another beautiful city which is located in the northwest part of greece'. So well done for that specific example!

### Third example with Cloud Computing


In [ ]:
context = """Cloud computing is the on-demand availability of computer system resources, especially data storage (cloud storage) and computing power, without direct 
             active management by the user. The term is generally used to describe data centers available to many users over the Internet.Large clouds, predominant today, 
             often have functions distributed over multiple locations from central servers. If the connection to the user is relatively close, it may be designated an edge
             server. Clouds may be limited to a single organization (enterprise clouds), or be available to multiple organizations (public cloud).Cloud computing relies 
             on sharing of resources to achieve coherence and economies of scale.Advocates of public and hybrid clouds note that cloud computing allows companies to avoid or 
             minimize up-front IT infrastructure costs. Proponents also claim that cloud computing allows enterprises to get their applications up and running faster, with 
             improved manageability and less maintenance, and that it enables IT teams to more rapidly adjust resources to meet fluctuating and unpredictable demand, providing 
             the burst computing capability: high computing power at certain periods of peak demand.Cloud providers typically use a "pay-as-you-go" model, which can lead
             to unexpected operating expenses if administrators are not familiarized with cloud-pricing models.The availability of high-capacity networks, low-cost computers
             and storage devices as well as the widespread adoption of hardware virtualization, service-oriented architecture and autonomic and utility computing has led to
             growth in cloud computing.By 2019, Linux was the most widely used operating system, including in Microsoft's offerings and is thus described as dominant.
          """

questions = ["What is cloud computing?",
             "What does cloud computing allow?",
             "What has led to growth in cloud computing?",
             ]

answers =   ["on-demand availability of computer system resources",
             "enterprises to get their applications up and running faster",
             """The availability of high-capacity networks, low-cost computers and storage devices as well as the widespread adoption of hardware virtualization, service-oriented architecture and autonomic and utility computing""",
             ]

for q,a in zip(questions,answers):
  query_answer(context,q,a)

Question: What is cloud computing?
Prediction: on - demand availability of computer system resources
True Answer: on-demand availability of computer system resources
EM: 0 	 F1: 0.7692307692307692

Question: What does cloud computing allow?
Prediction: enterprises to get their applications up and running faster
True Answer: enterprises to get their applications up and running faster
EM: 1 	 F1: 1.0

Question: What has led to growth in cloud computing?
Prediction: the availability of high - capacity networks, low - cost computers and storage devices as well as the widespread adoption of hardware virtualization, service - oriented architecture and autonomic and utility computing
True Answer: The availability of high-capacity networks, low-cost computers and storage devices as well as the widespread adoption of hardware virtualization, service-oriented architecture and autonomic and utility computing
EM: 0 	 F1: 0.6666666666666667



> Here we can clearly notice that fine tuned large bert model did amazing work and managed to find the right-appropriate answer to our questions by searching more accurately our more complex structured sentences! So fine tuned large bert model did what our model that we fine tuned couldn't handle! F1 scores are a bit tricky cause there aren't 1.0 in first and third example, due to fact that bert model manipulates in such way words like on-demand and transforms them to on - demand for example.But keep in mind that also this is a more complex model which produces embeddings with size 1024.